In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KDTree
from scores import scores
import time
import warnings
warnings.filterwarnings("ignore")

### 4.) Import Data

In [2]:
%run ../notebooks/utils.ipynb

In [3]:
trainandknn_Xy_original_df = pd.read_csv("../data/train_new.csv", sep="|")
train_Xy_original_df, knn_Xy_original_df = train_test_split(trainandknn_Xy_original_df,train_size=0.75, random_state=42) # if FINAL_SUBMISSION else 0.8) #small
test_X_original_df  = pd.read_csv("../data/test.csv", sep="|").iloc[0:500] #TODO: For faster testing we use less data from the test set
test_final_X_df = pd.read_csv("../data/test.csv", sep="|")

train_Xy_wo_knn_df = pd.read_csv("../data/train_new.csv", sep="|")

#Only for test routines
val_Xy_original_df = pd.read_csv("../data/val_new.csv", sep="|")
train_complete_Xy_original_df = pd.read_csv("../data/train.csv", sep="|")

In [4]:
#convention for variables names: datasetname_columntype_transformstatus_dataframeornot
train_y_original_df = train_Xy_original_df[["fraud"]].copy()
train_X_original_df = train_Xy_original_df.copy().drop("fraud", axis=1)

knn_y_original_df = knn_Xy_original_df[["fraud"]].copy()
knn_X_original_df = knn_Xy_original_df.copy().drop("fraud", axis=1)

# Only for test routie#nes
val_y_originial_df = val_Xy_original_df[["fraud"]].copy()
val_X_originial_df = val_Xy_original_df.copy().drop("fraud", axis=1)

train_y_wo_knn_df = train_Xy_wo_knn_df[["fraud"]].copy()
train_X_wo_knn_df = train_Xy_wo_knn_df.copy().drop("fraud", axis=1)

train_complete_y_originial_df = train_complete_Xy_original_df[["fraud"]].copy()
train_complete_X_originial_df = train_complete_Xy_original_df.copy().drop("fraud", axis=1)

In [5]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
transformer = DataTransformer(scaler)

# Adding new Features to train and test set
train_X_unscaled_df = transformer.add_features(train_X_original_df)
test_X_unscaled_df = transformer.add_features(test_X_original_df)
knn_X_unscaled_df = transformer.add_features(knn_X_original_df)

val_X_unscaled_df = transformer.add_features(val_X_originial_df)
train_complete_X_unscaled_df = transformer.add_features(train_complete_X_originial_df) 

transformer.fit_scaler(transformer.add_features(train_complete_X_unscaled_df.append(test_X_unscaled_df, sort=False)))
train_X_scaled_df = transformer.apply_scaler(train_X_unscaled_df)
knn_X_scaled_df   = transformer.apply_scaler(knn_X_unscaled_df)


test_X_scaled_df  = transformer.apply_scaler(test_X_unscaled_df)
val_X_scaled_df = transformer.apply_scaler(val_X_unscaled_df)
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df)

train_X_wo_knn_unscaled_df = transformer.add_features(train_X_wo_knn_df.copy())
train_X_wo_knn_scaled_df = transformer.apply_scaler(train_X_wo_knn_unscaled_df)

# labels
train_y_df = train_y_original_df.copy()
val_y_df = val_y_originial_df.copy()
knn_y_df = knn_y_original_df.copy()

train_complete_X_unscaled_df = transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud']))
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df.copy())
train_complete_y_df = train_complete_Xy_original_df.copy().fraud

test_final_X_df = transformer.add_features(test_final_X_df)


display(train_complete_X_unscaled_df.head(5))
display(train_complete_y_df.head(5))

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,29.0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,14.0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,13.0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,29.0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,27.0


0    0
1    0
2    0
3    0
4    0
Name: fraud, dtype: int64

## Scoring functions
Defining multiple scores which should be tracked in the HyperParamSearch Object

In [6]:
score = scores.Scores()

In [7]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}

## Defining the paramteres which should be tuned
To tune the hyperparameters, i looked into the [documentation here](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster). `randint` can be used for integer values, for float values, use `uniform`. 

You can also use a Grid search on single parameters to get a feeling for a good interval. If you want to try only two possibilities, you can create a list like for the `scale_pos_weight` parameter.

**Note: For the classifiers which work without gpu support, you can probably set a parameter n_jobs=-1 to use all processors**

In [8]:
params = {
    "max_depth": randint(3, 6),  # default 3
    "n_estimators": randint(300, 450),  # default 100
    "learnin_rate" :  uniform(0.1, 0.7),
    "gamma" : uniform(0,2),
    "min_child_weight" : uniform(0,50),
    "max_delta_step" : uniform(0,10), #Set it to value of 1-10 might help control the update.
    "reg_lambda" : uniform(0.1,2), 
    "num_parallel_tree" : [1, 20, 50, 100]    
    
}

## Creating a classifier with some default values
Not all paramters of a classifier should be fine tuned. For SVM for example, the `kernel`-paramter should be set manually. In the case of xgboost, some things like the objective, the booster and the tree method should not be tuned. The choice of paramters depend on the specific classifier

In [11]:
from xgboost import XGBClassifier
default_xgb = XGBClassifier(booster="gbtree",tree_method='gpu_hist',objective='binary:logistic',eval_metric='aucpr', n_gpus=1, verbosity=2)


In [12]:
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=30,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')
search.fit(train_complete_X_unscaled_df, train_complete_y_df)
results = search.cv_results_

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [ ]:
train_complete_Xy_unscaled_df = train_complete_X_unscaled_df.copy()
train_complete_Xy_unscaled_df['fraud'] =train_complete_y_df.copy()

train_Xy_wo_knn_unscaled_df = train_X_wo_knn_unscaled_df.copy()
train_Xy_wo_knn_unscaled_df['fraud'] = train_y_wo_knn_df

val_Xy_unscaled = val_X_unscaled_df.copy()
val_Xy_unscaled['fraud'] = val_y_df.copy()

In [ ]:
scorings = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}
xgbo = search.best_estimator_
result_dict = test_classification(xgbo,df_train=train_Xy_wo_knn_unscaled_df, df_val=val_Xy_unscaled)

In [ ]:
dmc = np.mean(search.cv_results_['mean_test_DMC'])
ap = np.mean(search.cv_results_['mean_test_AP'])
precision = np.mean(search.cv_results_['mean_test_Precision'])
recall = np.mean(search.cv_results_['mean_test_Recall'])

In [ ]:
print("DMC: {} --- Average Precision: {} --- Precision: {} --- Recall: {}".format(dmc, ap, precision, recall))

In [ ]:
search.best_estimator_

In [ ]:
search.best_params_